In [29]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [30]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow())
print(f"{current_date}")

2025-02-27 04:56:29.716355


In [31]:
type(current_date)

pandas._libs.tslibs.timestamps.Timestamp

In [32]:
from datetime import datetime, timedelta, timezone
current_date = pd.to_datetime(datetime.now(timezone.utc)).floor("h")

In [33]:
current_date.to_datetime64()

numpy.datetime64('2025-02-27T04:00:00.000000000')

In [34]:
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(weeks=52)

In [35]:
fetch_data_to

Timestamp('2025-02-27 04:00:00+0000', tz='UTC')

In [36]:
fetch_data_from

Timestamp('2024-02-29 04:00:00+0000', tz='UTC')

In [37]:
from datetime import datetime, timedelta
from typing import Union
import pandas as pd
from src.data_utils import load_and_process_taxi_data

def fetch_batch_raw_data(from_date: Union[datetime, str], to_date: Union[datetime, str]) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e., 1 year).

    Args:
        from_date (datetime or str): The start date for the data batch.
        to_date (datetime or str): The end date for the data batch.

    Returns:
        pd.DataFrame: A DataFrame containing the simulated production data.
    """
    # Convert string inputs to datetime if necessary
    if isinstance(from_date, str):
        from_date = datetime.fromisoformat(from_date)
    if isinstance(to_date, str):
        to_date = datetime.fromisoformat(to_date)

    # Validate input dates
    if not isinstance(from_date, datetime) or not isinstance(to_date, datetime):
        raise ValueError("Both 'from_date' and 'to_date' must be datetime objects or valid ISO format strings.")
    if from_date >= to_date:
        raise ValueError("'from_date' must be earlier than 'to_date'.")

    # Shift dates back by 52 weeks (1 year)
    historical_from_date = from_date - timedelta(weeks=52)
    historical_to_date = to_date - timedelta(weeks=52)

    print(historical_from_date)
    print(historical_to_date)

    # Load and filter data for the historical period
    rides_from = load_and_process_taxi_data(year=historical_from_date.year, months=[historical_from_date.month])
    rides_from = rides_from[rides_from.pickup_datetime >= historical_from_date.to_datetime64()]

    if historical_to_date.month != historical_from_date.month:
        rides_to = load_and_process_taxi_data(year=historical_to_date.year, months=[historical_to_date.month])
        rides_to = rides_to[rides_to.pickup_datetime < historical_to_date.to_datetime64()]
        # Combine the filtered data
        rides = pd.concat([rides_from, rides_to], ignore_index=True)
    else:
        rides = rides_from
    # Shift the data forward by 52 weeks to simulate recent data
    rides['pickup_datetime'] += timedelta(weeks=52)
    print(rides['pickup_datetime'])

    # Sort the data for consistency
    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [38]:
rides = fetch_batch_raw_data(fetch_data_from, fetch_data_to)

2023-03-02 04:00:00+00:00
2024-02-29 04:00:00+00:00
File already exists for 2023-03.
Loading data for 2023-03...
Total records: 3,403,766
Valid records: 3,331,705
Records dropped: 72,061 (2.12%)
Successfully processed data for 2023-03.
Combining all monthly data...
Data loading and processing complete!
File already exists for 2024-02.
Loading data for 2024-02...
Total records: 3,007,526
Valid records: 2,954,709
Records dropped: 52,817 (1.76%)
Successfully processed data for 2024-02.
Combining all monthly data...
Data loading and processing complete!
0         2024-03-05 14:07:12
1         2024-03-05 14:37:50
2         2024-03-05 14:57:46
3         2024-03-05 18:30:17
4         2024-03-05 20:28:25
                  ...        
6045057   2025-02-27 03:39:18
6045058   2025-02-27 03:56:53
6045059   2025-02-27 03:16:09
6045060   2025-02-27 03:15:20
6045061   2025-02-27 03:14:26
Name: pickup_datetime, Length: 6045062, dtype: datetime64[us]


In [39]:
rides.head(5)

,pickup_datetime,pickup_location_id
579989,2024-03-05 17:32:18,2
2823975,2024-03-27 10:17:09,2
3104107,2024-03-29 19:59:30,2
4499628,2025-02-12 16:25:44,2
4806329,2025-02-15 16:56:40,2


In [40]:
from src.data_utils import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

In [41]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2024-02-29 04:00:00,2,0
1,2024-02-29 05:00:00,2,0
2,2024-02-29 06:00:00,2,0
3,2024-02-29 07:00:00,2,0
4,2024-02-29 08:00:00,2,0
...,...,...,...
2253883,2025-02-26 23:00:00,263,44
2253884,2025-02-27 00:00:00,263,28
2253885,2025-02-27 01:00:00,263,17
2253886,2025-02-27 02:00:00,263,2


In [42]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2253888 entries, 0 to 2253887
Data columns (total 3 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pickup_hour         datetime64[ns]
 1   pickup_location_id  int16         
 2   rides               int16         
dtypes: datetime64[ns](1), int16(2)
memory usage: 25.8 MB


In [43]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time series data at hourly frequency",
    primary_key=['pickup_location_id','pickup_hour'],
    event_time = ['pickup_hour']
)

2025-02-26 23:57:27,841 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-26 23:57:27,868 INFO: Initializing external client
2025-02-26 23:57:27,869 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-26 23:57:28,723 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214648


In [44]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |███████████████████████████████| Rows 2253888/2253888 | Elapsed Time: 01:49 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1214648/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)